This script reads and saves all US POI data

In [1]:
import requests
import pandas as pd
import sys
from io import BytesIO
import gzip
from tqdm import tqdm
import io
import os
from global_variables import *

In [2]:
poi_csv_path = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_poi_with_store_id.csv.gz'
poi = pd.read_csv(poi_csv_path) 

In [17]:
output_path_parks = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_parks.csv.gz'

output_path_hospitals = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_hospitals.csv.gz'

output_path_pharmacies = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_pharmacies.csv.gz'

In [4]:
parks = poi[poi['SUB_CATEGORY']== "Nature Parks and Other Similar Institutions"]

with gzip.open(output_path_parks, 'wt', encoding='utf-8') as gzipped_file:
    parks.to_csv(gzipped_file, index=False)

In [11]:
hospitals = poi[poi['SUB_CATEGORY'].isin(['Offices of Physicians (except Mental Health Specialists)','General Medical and Surgical Hospitals','Specialty (except Psychiatric and Substance Abuse) Hospitals','Outpatient Care Centers']) | poi['TOP_CATEGORY'].isin(['Outpatient Care Centers'])]

with gzip.open(output_path_hospitals, 'wt', encoding='utf-8') as gzipped_file:
    hospitals.to_csv(gzipped_file, index=False)

In [14]:
pharmacies = poi[poi['SUB_CATEGORY'].isin(['Pharmacies and Drug Stores'])]

with gzip.open(output_path_pharmacies, 'wt', encoding='utf-8') as gzipped_file:
    pharmacies.to_csv(gzipped_file, index=False)

In [18]:
parks = pd.read_csv(output_path_parks)
hospitals = pd.read_csv(output_path_hospitals)
pharmacies = pd.read_csv(output_path_pharmacies)

In [19]:
date_start = '2023-01-01'
date_end = '2024-01-31'

file_suffix = date_start.replace('-', '')+"_"+date_end.replace('-', '')
file_suffix

'20230101_20240131'

In [ ]:
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/parks_hospitals_pharmacies_ft_data'+file_suffix+'.csv'
page = 1
download_count = 0
while True:
    print(page)
    results = requests.get(url=MONTHLY_FT_PATTERNS_PRODUCT_API_PATH,
                       params={'page': page,
                               'partition_key_after': date_start,   # optionally set date value here
                               'partition_key_before': date_end}, # optionally set date value here
                       headers={'X-API-KEY': API_KEY,
                                'accept': 'application/json'
                               })
    response_json = results.json()

    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
    
        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)
    
        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        filtered_df = df[df['PLACEKEY'].isin(parks['PLACEKEY']) |
                         df['PLACEKEY'].isin(hospitals['PLACEKEY']) |
                         df['PLACEKEY'].isin(pharmacies['PLACEKEY'])  ]
    
        selected_columns = ['PLACEKEY', 'LOCATION_NAME', 'STREET_ADDRESS','POSTAL_CODE','REGION','DATE_RANGE_START','DATE_RANGE_END','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','VISITS_BY_DAY','DISTANCE_FROM_HOME','MEDIAN_DWELL']
        filtered_df = filtered_df[selected_columns]
        
        if os.path.isfile(ft_data_path):
            header_option = False  
        else:
            header_option = True  
    
        filtered_df.to_csv(ft_data_path, mode='a', header=header_option, index=False)
        
        download_count += 1
        
    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1   

In [22]:
df = pd.read_csv(ft_data_path)  

with gzip.open(ft_data_path+".gz", 'wt', encoding='utf-8') as gzipped_file:
    df.to_csv(gzipped_file, index=False)

C:\Users\dratnadiwakara2\AppData\Local\Temp\ipykernel_27448\2720968004.py:1: DtypeWarning: Columns (3,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ft_data_path)
